In [2]:
import numpy as np
import cv2
from PIL import Image
import os


In [3]:

print(np.version.version)


1.24.2


In [4]:

def appendImg(url,path,target):
    images=[]
    labels=[]
    for i in range(len(url)):
        image=path+url[i];
        
        img=cv2.imread(image)
        img=img/255
        if img is None:
            print()
        else:
            image=cv2.resize(img,(200,200))
        # image = img.resize((200,200))
        # image = img.resize(200,200)
        
        images.append(image)
        labels.append(target)
    images=np.asarray(images);
    labels=np.asarray(labels);
    
    
    return images,labels


In [5]:
turl=os.listdir("C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/brain_tumor_dataset/yes")
# turl1=os.listdir("C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/yes/yes")
tpath="C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/brain_tumor_dataset/yes/"
# tpath1="C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/yes/yes"
img=tpath+turl[0]
print(img)

len(turl)

C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/brain_tumor_dataset/yes/y0.jpg


1501

In [6]:
img=tpath+turl[0]
img=cv2.imread(img)
image=cv2.resize(img,(200,200))
image.shape

(200, 200, 3)

In [7]:
Nurl=os.listdir("C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/brain_tumor_dataset/no")
# Nurl1=os.listdir("C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/no/no")
Npath="C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/brain_tumor_dataset/no/"
# Npath1="C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/no/no"

len(Nurl)


1586

In [8]:
#images with tumour
tumourimage, ttarget=appendImg(turl,tpath,1)
# tumourimage, ttarget=appendImg(turl1,tpath1,1)
normalimage, ntarget=appendImg(Nurl,Npath,0)

In [9]:
data=np.r_[tumourimage,normalimage]
targets=np.r_[ttarget,ntarget]

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(data,targets,random_state=2,test_size=0.2)

In [11]:
y_train.shape
print(x_test.shape)


(618, 200, 200, 3)


In [12]:
import tensorflow as tf


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense

*Using Transfer learning for Tumor classification


In [13]:
from keras.applications.vgg16 import VGG16

In [14]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(200,200,3)
)

In [15]:
from tensorflow import keras


In [16]:
modelseq=Sequential()
modelseq.add(conv_base)
modelseq.add(Flatten())
modelseq.add(Dense(1024, activation='relu'))
modelseq.add( Dense(512,activation='relu'))
modelseq.add(Dense(1,activation='sigmoid'))

In [26]:
conv_base.trainable= False

In [32]:
modelseq.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [37]:
history = modelseq.fit(x_train,y_train,epochs=1,batch_size=16,validation_data=(x_test,y_test))

 64/155 [===========>..................] - ETA: 2:28 - loss: 0.6931 - accuracy: 0.5117

KeyboardInterrupt: 

In [ ]:
# y_predict=modelseq.predict(x_test[0])
# print(y_predict)
x_test[0].shape
xre=np.reshape(x_test[0],(1,200,200,3))

y_predict=modelseq.predict(x_test)
print(y_predict)
y_pred=np.where(y_predict>0.5,1,0)
y_test


10/20 [==============>...............] - ETA: 49s

In [ ]:
target=['safe','not safe']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

        safe       0.88      0.98      0.93       312
    not safe       0.98      0.86      0.92       306

    accuracy                           0.92       618
   macro avg       0.93      0.92      0.92       618
weighted avg       0.93      0.92      0.92       618



In [34]:
# model= Sequential([Conv2D(32,4,input_shape=(200,200,3), activation='relu'),
#                    MaxPooling2D(),
#                     Conv2D(16,4,input_shape=(200,200,3), activation='relu'),
#                     MaxPooling2D(),
#                     Conv2D(16,3,input_shape=(200,200,3), activation='relu'),
#                     MaxPooling2D(),
#                    Flatten(),
#                    Dense(128, activation='relu'),
#                    Dense(64,activation='relu'),
#                    Dense(1,activation='sigmoid')
               
#                    ])

model= Sequential([Conv2D(32,3,input_shape=(200,200,3), activation='relu'),
                   MaxPooling2D(),
                    Conv2D(16,3,input_shape=(200,200,3), activation='relu'),
                    MaxPooling2D(),
                    Conv2D(16,3,input_shape=(200,200,3), activation='relu'),
                    MaxPooling2D(),
                   Flatten(),
                   Dense(1024, activation='relu'),
                   Dense(512,activation='relu'),
                   Dense(1,activation='sigmoid')
               
     
     
     #Conv2D(32,3,input_shape=(200,200,3), activation='relu'),
#                    MaxPooling2D(),
#                     Conv2D(16,3,input_shape=(200,200,3), activation='relu'),
#                     MaxPooling2D(),
#                     Conv2D(16,3,input_shape=(200,200,3), activation='relu'),
#                     MaxPooling2D(),
#                    Flatten(),
#                    Dense(128, activation='relu'),
#                    Dense(64,activation='relu'),
#                    Dense(1,activation='softmax')

               
                   ])


In [35]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [36]:
model.fit(x_train,y_train,batch_size=16,epochs=4,validation_data=(x_test,y_test))

Epoch 1/4
 57/155 [==========>...................] - ETA: 33s - loss: 0.5398 - accuracy: 0.7346

KeyboardInterrupt: 

In [ ]:
model.save("tumour.h5")

In [ ]:
from keras.models import load_model

In [ ]:
keras_model=load_model('tumour.h5')

In [ ]:
#Model metrics

y_predict=model.predict(x_test)
print(y_predict)
y_pred=np.where(y_predict>0.5,1,0)
y_test


20/20 [==============================] - 3s 120ms/step
[[1.00000000e+00]
 [9.99999821e-01]
 [9.99996305e-01]
 [1.60215393e-06]
 [1.83533892e-01]
 [3.00958618e-08]
 [9.99999881e-01]
 [1.00000000e+00]
 [9.99807358e-01]
 [1.21350206e-01]
 [9.99994755e-01]
 [9.99198735e-01]
 [9.74790096e-01]
 [9.99999940e-01]
 [2.68576166e-08]
 [9.99995470e-01]
 [9.99995172e-01]
 [4.84457389e-02]
 [9.98477161e-01]
 [9.99914944e-01]
 [9.99997973e-01]
 [3.93109940e-06]
 [9.83712733e-01]
 [5.33762157e-01]
 [1.00000000e+00]
 [3.32964701e-03]
 [2.14658612e-05]
 [5.08586818e-05]
 [3.18714930e-03]
 [6.87765307e-04]
 [9.63919272e-04]
 [9.99999642e-01]
 [1.00000000e+00]
 [9.39389211e-05]
 [9.99999762e-01]
 [3.84331477e-04]
 [7.62718124e-03]
 [9.93336737e-01]
 [9.92301285e-01]
 [9.99999583e-01]
 [9.53654060e-04]
 [5.80270290e-01]
 [9.99346256e-01]
 [9.99999106e-01]
 [8.74311328e-01]
 [9.14105203e-06]
 [2.02549193e-11]
 [9.99998331e-01]
 [9.99993026e-01]
 [9.99787688e-01]
 [6.09165020e-02]
 [9.99999940e-01]
 [6.64288

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       333
           1       0.82      1.00      0.90       285

    accuracy                           0.90       618
   macro avg       0.91      0.91      0.90       618
weighted avg       0.92      0.90      0.90       618



In [ ]:
#Making a predictive system
input_image="C:/Users/rnegi/Desktop/ML complete/ML VS code/tumourDetection/yes/Y8.jpg"
img=cv2.imread(input_image)
img=cv2.resize(img,(200,200))
imgre=np.reshape(img,[1,200,200,3])
prediction=keras_model.predict(imgre)
print(prediction)

[[0.8051845]]


In [ ]:
print(np.version.version)

1.24.2


In [ ]:
import pandas as pd




In [ ]:


print('hello')






hello


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from sklearn.metrics import classification_report
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.50      1.00      0.67         1
     class 1       0.00      0.00      0.00         1
     class 2       1.00      0.67      0.80         3

    accuracy                           0.60         5
   macro avg       0.50      0.56      0.49         5
weighted avg       0.70      0.60      0.61         5

